<a href="https://colab.research.google.com/github/Rohithtechis/Anime_recommendation/blob/main/anime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
anime = pd.read_csv('/content/anime.csv')
rating = pd.read_csv('/content/rating.csv')

In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [3]:
anime.shape

(12294, 7)

In [4]:
rating.shape

(7813737, 3)

In [7]:
anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [8]:
anime.fillna({'rating':0},inplace=True)

In [14]:
rating.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [13]:
print('Anime info', anime.info())
print('*******')
print('Rating info', rating.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None
*******
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


In [15]:
anime=anime[~np.isnan(anime["rating"])]

# filling mode value for genre and type
anime['genre'] = anime['genre'].fillna(
anime['genre'].dropna().mode().values[0])

anime['type'] = anime['type'].fillna(
anime['type'].dropna().mode().values[0])

#checking if all null values are filled
anime.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [17]:
rating['rating'] = rating['rating'].apply(lambda x: np.nan if x==-1 else x)
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [19]:
#step 1
anime = anime[anime['type']=='TV']

#step 2
rated_anime = rating.merge(anime, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])

#step 3
rated_anime =rated_anime[['user_id', 'name', 'rating']]

#step 4
rated_anime_7500= rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()

,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


In [20]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,A-Channel,A.D. Police (TV),AKB0048,AKB0048: Next Stage,Aa! Megami-sama! (TV),Aa! Megami-sama!: Chichaitte Koto wa Benri da ne,Aa! Megami-sama!: Sorezore no Tsubasa,Abarenbou Rikishi!! Matsutarou,Abenobashi Mahou☆Shoutengai,Absolute Duo,Accel World,Acchi Kocchi (TV),Ace wo Nerae!,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari 2nd,Afro Samurai,After War Gundam X,Agatha Christie no Meitantei Poirot to Marple,Ai Shoujo Pollyanna Story,Ai Tenchi Muyou!,Ai Tenshi Densetsu Wedding Peach,Ai Yori Aoshi,Ai Yori Aoshi: Enishi,Ai no Gakko Cuore Monogatari,Ai no Wakakusa Monogatari,Ai to Yuuki no Pig Girl Tonde Buurin,Aikatsu!,Air,Air Gear,Air Master,...,Yu☆Gi☆Oh! 5D&#039;s,Yu☆Gi☆Oh! Duel Monsters,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh! Zexal Second,Yu☆Gi☆Oh!: Duel Monsters GX,Z/X: Ignition,Zan Sayonara Zetsubou Sensei,Zankyou no Terror,Zegapain,Zenryoku Usagi,Zero Duel Masters,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zetman,Zetsuen no Tempest,Zettai Bouei Leviathan,Zettai Karen Children,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Muteki Raijin-Oh,Zettai Shounen,Zipang,Zoids,Zoids Fuzors,Zoids Genesis,Zoids Shinseiki/Zero,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.78,7.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.62,NaN,NaN,NaN,NaN,7.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.69,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.78,7.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.57,NaN,NaN,7.23,NaN,8.01,NaN,NaN,NaN,NaN,7.62,7.77,7.73,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [22]:
import scipy as sp
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

pivot_n.fillna(0, inplace=True)

pivot_n = pivot_n.T

pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
anime_similarity = cosine_similarity(piv_sparse)

ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [25]:
ani_sim_df.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,A-Channel,A.D. Police (TV),AKB0048,AKB0048: Next Stage,Aa! Megami-sama! (TV),Aa! Megami-sama!: Chichaitte Koto wa Benri da ne,Aa! Megami-sama!: Sorezore no Tsubasa,Abarenbou Rikishi!! Matsutarou,Abenobashi Mahou☆Shoutengai,Absolute Duo,Accel World,Acchi Kocchi (TV),Ace wo Nerae!,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari,Active Raid: Kidou Kyoushuushitsu Dai Hachi Gakari 2nd,Afro Samurai,After War Gundam X,Agatha Christie no Meitantei Poirot to Marple,Ai Shoujo Pollyanna Story,Ai Tenchi Muyou!,Ai Tenshi Densetsu Wedding Peach,Ai Yori Aoshi,Ai Yori Aoshi: Enishi,Ai no Gakko Cuore Monogatari,Ai no Wakakusa Monogatari,Ai to Yuuki no Pig Girl Tonde Buurin,Aikatsu!,Air,Air Gear,Air Master,...,Yu☆Gi☆Oh! 5D&#039;s,Yu☆Gi☆Oh! Duel Monsters,Yu☆Gi☆Oh! Zexal,Yu☆Gi☆Oh! Zexal Second,Yu☆Gi☆Oh!: Duel Monsters GX,Z/X: Ignition,Zan Sayonara Zetsubou Sensei,Zankyou no Terror,Zegapain,Zenryoku Usagi,Zero Duel Masters,Zero no Tsukaima,Zero no Tsukaima F,Zero no Tsukaima: Futatsuki no Kishi,Zero no Tsukaima: Princesses no Rondo,Zetman,Zetsuen no Tempest,Zettai Bouei Leviathan,Zettai Karen Children,Zettai Karen Children: The Unlimited - Hyoubu Kyousuke,Zettai Muteki Raijin-Oh,Zettai Shounen,Zipang,Zoids,Zoids Fuzors,Zoids Genesis,Zoids Shinseiki/Zero,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,1.000000,0.464082,0.445687,0.025392,0.069076,0.117115,0.005797,0.000000,0.056980,-0.023306,0.042509,0.076374,0.015547,-0.040314,0.092177,0.081833,-0.045462,0.029569,0.034892,0.058461,0.027262,-0.014617,0.021660,0.017629,0.018486,0.066619,0.058972,0.004531,0.000000,0.012443,0.016530,0.053239,0.046128,0.0,0.022523,0.006841,0.0,0.075906,0.000677,0.057638,...,0.010886,0.034128,0.016592,0.020508,0.060860,0.033495,-0.093330,-0.070570,0.023236,0.020983,0.017704,0.009560,-0.049050,-0.059634,0.008698,0.068171,-0.128398,0.064709,0.048660,0.016697,0.000000,0.119957,-0.003809,0.100148,0.027114,0.033444,0.079001,-0.110971,-0.113270,0.096986,0.077054,0.0,-0.112560,-0.109070,0.000000,0.000000,0.071199,0.125517,-0.137733,-0.112823
.hack//Sign,0.464082,1.000000,0.443120,0.031732,0.093941,0.090260,0.004336,0.000000,0.069243,-0.024904,0.042135,0.042904,0.025741,-0.037211,0.139368,0.074214,-0.023978,0.003661,0.085503,0.049467,0.052258,0.010528,0.018297,0.031459,0.012013,0.094886,0.073792,0.004039,0.000000,0.008108,0.016927,0.086613,0.088499,0.0,0.004062,0.016157,0.0,0.126247,0.034256,0.049150,...,0.015994,0.065764,0.027197,0.012201,0.056831,0.025462,-0.100347,-0.091101,0.028220,0.011432,0.024986,0.062998,-0.037885,-0.028702,0.039345,0.065810,-0.109224,0.054043,0.050155,0.017713,0.007578,0.084014,0.023953,0.121234,0.044780,0.044682,0.139842,-0.128195,-0.115278,0.090533,0.085169,0.0,-0.112569,-0.118462,0.000000,0.000000,0.036096,0.149564,-0.132065,-0.117723
.hack//Tasogare no Udewa Densetsu,0.445687,0.443120,1.000000,0.061698,0.057993,0.091261,0.007159,0.000000,0.049910,-0.023922,0.048995,0.070165,0.009600,-0.059430,0.082728,0.109934,-0.067293,0.000000,0.056110,0.044227,0.024814,-0.008986,0.019569,0.015671,0.005902,0.054761,0.070319,0.000000,0.000000,0.014594,0.028414,0.069397,0.076973,0.0,0.005653,0.007884,0.0,0.058689,-0.010874,0.021644,...,0.003167,0.026672,0.024170,0.014739,0.049874,0.029706,-0.104591,-0.073108,0.024419,0.025636,0.013279,0.004955,-0.048020,-0.064660,0.013323,0.066735,-0.107386,0.054044,0.034351,0.005686,0.007733,0.028372,0.000992,0.087697,0.052725,0.028381,0.107020,-0.107215,-0.112890,0.093319,0.037200,0.0,-0.130604,-0.134231,0.000000,0.000000,0.06

In [26]:
def anime_recommendation(ani_name):
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1

In [32]:
anime_recommendation('Dragon Ball')

Recommended because you watched Dragon Ball:

#1: Dragon Ball Z, 79.32% match
#2: Fullmetal Alchemist, 39.96% match
#3: Death Note, 38.22% match
#4: Digimon Adventure, 36.12% match
#5: Yuu☆Yuu☆Hakusho, 34.88% match
